In [2]:
import json, pathlib, pandas as pd
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import os, csv
import cProfile, pstats
from persistent_homology_c import (
    BettiZero,
    generate_sphere_points,
)

In [3]:
from persistent_homology import compute_n_largest_bars,     compute_intervals

In [8]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
    
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]

def load_vertices_edges(seg_folder):
    """Read vertices/edges only once per segmentation."""
    folder_path = '../lung_segmentations/{}'.format(seg_folder)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    verts = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    return verts, edges

def process_direction(args):
    """Run β₀ persistence for one direction (runs in worker)."""
    direction, vertices, edges = args
    bz = BettiZero(direction, vertices, edges)
    comps, mergers, verts, births = bz.compute()
    intervals = compute_intervals(births, mergers)
    bars = compute_n_largest_bars(intervals,5)
    return {
        "direction": list(direction),
        "intervals": intervals,
        "largest_bars": bars,
        #"largest_length": length,
        "components": list(comps),
    }

In [9]:
json_data = {}

segmentation_folder_path = "../lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

In [10]:
vertices, edges = load_vertices_edges('Lung segmentation-10')
directions = generate_sphere_points(2, 2, 1e-7)

In [11]:
process_direction(([0,0,1], vertices, edges))

TypeError: list indices must be integers or slices, not str